# Objectif du script:
- Visualiser sur une cartographie les prix au m² par commune

In [1]:
import pandas as pd
import numpy as np

# Lecture du dataframe Valeurs Foncieres

In [2]:
df = pd.read_csv('databases/inter/03C - dvf_merged_2020.csv', 
                 dtype = {'Code departement': 'str', 'Code commune': 'str', "Code postal": 'str',
                         'Code voie': 'str', 'No Volume': 'str', '1er lot': 'str', '2eme lot': 'str', '3eme lot': 'str', 
                         'Nature culture speciale': 'str', 'Code postal 5 chiffres': 'str', 'Code commune INSEE': 'str'})
df.head()

,Unnamed: 0,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,...,Population totale,SNHMO18,SNHMFO18,dep,anciennete,consommation_energie,classe_consommation_energie,estimation_ges,classe_estimation_ges,surface_baies_orientees_sud
0,0,1,09/01/2020,Vente,72000.0,5367.0,NaN,NaN,B107,LORETTE,...,NaN,11.531505,10.275121,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,06/01/2020,Vente,180300.0,31.0,NaN,RUE,0970,COMTE DE LA TEYSSONNIERE,...,NaN,11.531505,10.275121,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,06/01/2020,Vente,54800.0,6.0,NaN,AV,0770,DU CHAMP DE FOIRE,...,NaN,11.531505,10.275121,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,13/01/2020,Vente,97000.0,16.0,NaN,RUE,1770,DES GRAVES,...,NaN,11.531505,10.275121,1.0,42.0,267.37,E,7.22,B,0.0
4,4,1,09/01/2020,Vente,215000.0,2.0,NaN,IMP,0140,DU COLOMBIER,...,NaN,11.531505,10.275121,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns

Index(['Unnamed: 0', 'No disposition', 'Date mutation', 'Nature mutation',
       'Valeur fonciere', 'No voie', 'B/T/Q', 'Type de voie', 'Code voie',
       'Voie', 'Code postal', 'Commune', 'Code departement', 'Code commune',
       'Prefixe de section', 'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain', 'Code commune INSEE',
       'Ville', 'Adresse', 'Prix m2', 'Paris', 'dep_name', 'region_name',
       'Mer', 'Commune_Lat', 'Commune_Lon', 'lon', 'lat',
       'Code postal 5 chiffres', 'code_iris_clean', 'Alimentation', 'Aéroport',
       'Baignade', 'Banques', 'Boulan

In [4]:
# Suppression des colonnes inutiles

df = df.drop(columns = {'Unnamed: 0', 'No disposition', 'Nature mutation', 'Nature culture', 'Nature culture speciale', 
                        'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Prefixe de section', 
                   'Section', 'No plan', 'No Volume', '1er lot', 'Surface Carrez du 1er lot', '2eme lot', 
                        'Surface Carrez du 2eme lot', '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot', 
                        'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot', 'Nombre de lots',
                       'Identifiant local', 'Code postal'})

df.columns

Index(['Date mutation', 'Valeur fonciere', 'Commune', 'Code departement',
       'Code commune', 'Code type local', 'Type local', 'Surface reelle bati',
       'Nombre pieces principales', 'Surface terrain', 'Code commune INSEE',
       'Ville', 'Adresse', 'Prix m2', 'Paris', 'dep_name', 'region_name',
       'Mer', 'Commune_Lat', 'Commune_Lon', 'lon', 'lat',
       'Code postal 5 chiffres', 'code_iris_clean', 'Alimentation', 'Aéroport',
       'Baignade', 'Banques', 'Boulangerie', 'Camping', 'Cinéma',
       'Collège ou lycée', 'Crèche', 'Ecole mat et primaire', 'Ecole sup',
       'Etab hospi', 'Gare', 'Gare nationale', 'Hotels', 'Info touristique',
       'Parcours sportif/santé', 'Pharmacie', 'Police ou gendarmerie',
       'Port plaisance', 'Restaurants', 'Resto U', 'Résidence U',
       'Salle multisport', 'loyer', 'Population totale', 'SNHMO18', 'SNHMFO18',
       'dep', 'anciennete', 'consommation_energie',
       'classe_consommation_energie', 'estimation_ges',
       'classe_

# Calcul du prix par m2 par commune

In [36]:
type_bien = "Tous"
prix_moyen_commune = df.groupby(['Code commune INSEE', 'Code postal 5 chiffres', 'Commune'])['Prix m2'].mean().reset_index()

In [37]:
prix_maison = df[df['Type local'] == "Maison"].groupby(['Code commune INSEE', 'Code postal 5 chiffres', 'Commune'])['Prix m2'].mean().reset_index()
prix_maison = prix_maison.rename(columns = {'Prix m2': 'Prix m2 maison'})

prix_appart = df[df['Type local'] == "Appartement"].groupby(['Code commune INSEE', 'Code postal 5 chiffres', 'Commune'])['Prix m2'].mean().reset_index()
prix_appart = prix_appart.rename(columns = {'Prix m2': 'Prix m2 appart'})

prix_moyen_commune = prix_moyen_commune.merge(prix_maison, on = ['Code commune INSEE', 'Code postal 5 chiffres', 'Commune'], how = "left")
prix_moyen_commune = prix_moyen_commune.merge(prix_appart, on = ['Code commune INSEE', 'Code postal 5 chiffres', 'Commune'], how = "left")
prix_moyen_commune.head(50)

,Code commune INSEE,Code postal 5 chiffres,Commune,Prix m2,Prix m2 maison,Prix m2 appart
0,10003,10160,AIX-VILLEMAUR-PALIS,1455.567994,1455.567994,NaN
1,10003,10190,AIX-VILLEMAUR-PALIS,1646.718066,1646.718066,NaN
2,10005,10140,AMANCE,2166.666667,2166.666667,NaN
3,10006,10700,ARCIS SUR AUBE,1519.318384,1519.318384,NaN
4,1001,01400,L'ABERGEMENT-CLEMENCIAT,1778.409091,1778.409091,NaN
5,10012,10200,ARSONVAL,1756.944444,1756.944444,NaN
6,10014,10220,ASSENCIERES,1382.065863,1382.065863,NaN
7,10015,10150,AUBETERRE,1776.179162,1776.179162,NaN
8,10017,10240,AULNAY,1224.137931,1224.137931,NaN
9,10018,10130,AUXON,1237.885328,1237.885328,NaN


In [38]:
prix_moyen_commune[prix_moyen_commune['Commune'].str.startswith("PARIS")]

,Code commune INSEE,Code postal 5 chiffres,Commune,Prix m2,Prix m2 maison,Prix m2 appart
16660,71343,71150,PARIS-L HOPITAL,1334.158286,1334.158286,NaN
17739,75101,75001,PARIS,12981.544790,NaN,12981.544790
17740,75103,75003,PARIS,12991.142620,NaN,12991.142620
17741,75105,75005,PARIS,12907.422128,NaN,12907.422128
17742,75106,75006,PARIS,14542.367300,17500.000000,14535.838309
17743,75107,75007,PARIS,14333.391904,NaN,14333.391904
17744,75108,75008,PARIS,12636.667366,NaN,12636.667366
17745,75109,75009,PARIS,12095.787673,NaN,12095.787673
17746,75110,75010,PARIS,11071.174172,NaN,11071.174172
17747,75111,75011,PARIS,11321.051340,13562.275180,11318.733632


# Calcul de la cartographie

In [39]:
import folium
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

In [40]:
import geojson, geopandas

# lecture des données initiales
c = geopandas.read_file("databases/shapefiles/communes-20210101.shp")
c = c.drop(columns = {'wikipedia'})
c.head()

,insee,nom,surf_ha,geometry
0,30225,Sabran,3563.0,"POLYGON ((4.49703 44.14344, 4.49727 44.14398, ..."
1,26334,Salettes,706.0,"POLYGON ((4.94213 44.57481, 4.94283 44.57501, ..."
2,04200,Salignac,1435.0,"POLYGON ((5.95337 44.15613, 5.95341 44.15649, ..."
3,04179,Saint-Geniez,3900.0,"POLYGON ((5.98593 44.23280, 5.98654 44.23292, ..."
4,05162,La Saulce,771.0,"POLYGON ((5.98920 44.41892, 5.98921 44.41894, ..."


In [41]:
c.shape

(34969, 4)

In [42]:
# Ajout des arrondissements de Paris, Lyon et Marseille
a_all = geopandas.read_file("databases/shapefiles/arrondissements-millesimes0.shp", encoding='utf-8')
a_all = a_all[['code_insee', 'superficie', 'nom_com', 'geometry']]
a_all = a_all.rename(columns = {'code_insee' : 'insee', 'superficie': 'surf_ha', 'nom_com': 'nom'})
a_all = a_all.reindex(columns=['insee','nom','surf_ha','geometry'])

a_all.head()

,insee,nom,surf_ha,geometry
0,75106,PARIS-6E-ARRONDISSEMENT,215.0,"POLYGON ((2.31663 48.84675, 2.31996 48.84815, ..."
1,75118,PARIS-18E-ARRONDISSEMENT,604.0,"POLYGON ((2.36468 48.88429, 2.36352 48.88434, ..."
2,13205,MARSEILLE-5E-ARRONDISSEMENT,219.0,"POLYGON ((5.38908 43.30031, 5.38905 43.29890, ..."
3,75101,PARIS-1ER-ARRONDISSEMENT,182.0,"POLYGON ((2.32576 48.86955, 2.32767 48.86986, ..."
4,13202,MARSEILLE-2E-ARRONDISSEMENT,327.0,"POLYGON ((5.37362 43.29499, 5.37327 43.29580, ..."


In [43]:
# Concaténation des deux dataframes geojson
c = pd.concat([a_all, c], axis = 0, ignore_index=True)
c = c.rename(columns = {'insee' : 'DEPCOM'})
c.head()

,DEPCOM,nom,surf_ha,geometry
0,75106,PARIS-6E-ARRONDISSEMENT,215.0,"POLYGON ((2.31663 48.84675, 2.31996 48.84815, ..."
1,75118,PARIS-18E-ARRONDISSEMENT,604.0,"POLYGON ((2.36468 48.88429, 2.36352 48.88434, ..."
2,13205,MARSEILLE-5E-ARRONDISSEMENT,219.0,"POLYGON ((5.38908 43.30031, 5.38905 43.29890, ..."
3,75101,PARIS-1ER-ARRONDISSEMENT,182.0,"POLYGON ((2.32576 48.86955, 2.32767 48.86986, ..."
4,13202,MARSEILLE-2E-ARRONDISSEMENT,327.0,"POLYGON ((5.37362 43.29499, 5.37327 43.29580, ..."


In [44]:
# Suppression des codes communes correspondant à l'ensemble de Paris, Marseille et Lyon
c = c[c['DEPCOM'] != '13055']
c = c[c['DEPCOM'] != '69123']
c = c[c['DEPCOM'] != '75056']

In [45]:
# Ajout du prix_m2
prix_moyen_commune = prix_moyen_commune.rename(columns = {'Code commune INSEE': 'DEPCOM'})
c = c.merge(prix_moyen_commune, on = "DEPCOM", how = "left")
c.head()

,DEPCOM,nom,surf_ha,geometry,Code postal 5 chiffres,Commune,Prix m2,Prix m2 maison,Prix m2 appart
0,75106,PARIS-6E-ARRONDISSEMENT,215.0,"POLYGON ((2.31663 48.84675, 2.31996 48.84815, ...",75006,PARIS,14542.367300,17500.000000,14535.838309
1,75118,PARIS-18E-ARRONDISSEMENT,604.0,"POLYGON ((2.36468 48.88429, 2.36352 48.88434, ...",75018,PARIS,10587.670600,NaN,10587.670600
2,13205,MARSEILLE-5E-ARRONDISSEMENT,219.0,"POLYGON ((5.38908 43.30031, 5.38905 43.29890, ...",13005,MARSEILLE,2908.458387,4015.233390,2898.594082
3,75101,PARIS-1ER-ARRONDISSEMENT,182.0,"POLYGON ((2.32576 48.86955, 2.32767 48.86986, ...",75001,PARIS,12981.544790,NaN,12981.544790
4,13202,MARSEILLE-2E-ARRONDISSEMENT,327.0,"POLYGON ((5.37362 43.29499, 5.37327 43.29580, ...",13002,MARSEILLE,2848.008608,2866.071429,2847.861157


In [46]:
# Test Paris 17
c['DEPCOM'] = c['DEPCOM'].astype('str')
c[c['DEPCOM'] == '75117']

,DEPCOM,nom,surf_ha,geometry,Code postal 5 chiffres,Commune,Prix m2,Prix m2 maison,Prix m2 appart
14,75117,PARIS-17E-ARRONDISSEMENT,564.0,"POLYGON ((2.28446 48.88564, 2.28567 48.88657, ...",75017,PARIS,11601.246273,14698.703255,11594.908765


In [47]:
# Arrondi du prix
c['Prix m2'] = c['Prix m2'].round(0)

In [48]:
# Département choisi
dep = '33'
c_selec = c[c['DEPCOM'].str.startswith(dep)]
c_selec.head()

,DEPCOM,nom,surf_ha,geometry,Code postal 5 chiffres,Commune,Prix m2,Prix m2 maison,Prix m2 appart
972,33499,Les Salles-de-Castillon,1120.0,"POLYGON ((-0.02281 44.91542, -0.02281 44.91548...",33350,LES SALLES,1315.0,1315.384615,NaN
973,33499,Les Salles-de-Castillon,1120.0,"POLYGON ((-0.02281 44.91542, -0.02281 44.91548...",33350,LES SALLES DE CASTILLON,1361.0,1361.111111,NaN
981,33269,Margueron,1360.0,"POLYGON ((0.20983 44.76873, 0.20996 44.76902, ...",NaN,NaN,NaN,NaN,NaN
2262,33402,Sainte-Foy-la-Grande,57.0,"POLYGON ((0.21016 44.84169, 0.21087 44.84221, ...",33220,SAINTE-FOY-LA-GRANDE,1607.0,1644.242281,1232.608696
3878,33360,La Roquille,655.0,"POLYGON ((0.20626 44.78002, 0.20631 44.78032, ...",NaN,NaN,NaN,NaN,NaN


In [49]:
with open("c_selec.geojson", "w") as f:
    geojson.dump(c_selec, f)

In [50]:
# Import des coordonnées GPS par département pour centrer la cartographie
gps_dep = pd.read_csv('databases/GPS_departement.csv', sep = ";")
gps_dep.head()

,Code departement,Nom departement,Chef lieu,Longitude,Latitude
0,1,Ain,Bourg-en-Bresse,5.348889,46.099444
1,2,Aisne,Laon,3.558333,49.559444
2,3,Allier,Moulins,3.188333,46.393611
3,4,Alpes-de-Haute-Provence,Digne-les-Bains,6.243889,44.106111
4,5,Hautes-Alpes,Gap,6.263056,44.663611


In [51]:
dep_latitude = gps_dep[gps_dep['Code departement'] == dep]['Latitude']
dep_longitude = gps_dep[gps_dep['Code departement'] == dep]['Longitude']
dep_nom = gps_dep[gps_dep['Code departement'] == dep]['Nom departement']

In [53]:
if type_bien == "Maisons":
    fields = ['Commune', 'Prix m2 maison']
    columns = ['DEPCOM', 'Prix m2 maison']
elif type_bien == "Appartements":
    fields = ['Commune', 'Prix m2 appart']
    columns = ['DEPCOM', 'Prix m2 appart']
else:
    fields = ['Commune', 'Prix m2']
    columns = ['DEPCOM', 'Prix m2']

# Map

coords = (dep_latitude,dep_longitude)

f = folium.Figure(width=680, height=750)

#map = folium.Map([23.53, 78.3], maxZoom=6,minZoom=4.8,zoom_control=True,zoom_start=5,
#               scrollWheelZoom=True,maxBounds=[[40, 68],[6, 97]],attr='white tile',
#               dragging=True).add_to(f)

map = folium.Map(location=coords, tiles='OpenStreetMap', zoom_start=8).add_to(f)

popup = GeoJsonPopup(
    fields=fields,
    aliases=['Commune',"Prix au m² : "], 
    localize=True,
    labels=True,
    style="background-color: yellow;"
)

tooltip = GeoJsonTooltip(
    fields=fields,
    aliases=['Commune',"Prix au m² : "],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 1px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800
)

g = folium.Choropleth(
    geo_data = 'c_selec.geojson',
    name = 'choropleth',
    data = c_selec,
    columns = columns, # data key/value pair
    key_on = 'feature.properties.DEPCOM', # corresponding layer in GeoJSON
    threshold_scale = [0, 2000, 4000, 6000, 8000, 10000, 20000],
    fill_color = 'OrRd',
    fill_opacity = 0.8,
    line_opacity = 0.2,
    legend_name = 'Prix m2'
).add_to(map)

folium.GeoJson(
    'c_selec.geojson',
    style_function=lambda feature: {
        'fillColor': '#ffff00',
        'color': 'black',
        'weight': 0.2,
        'dashArray': '5, 5'
    },
    tooltip=tooltip,
    popup=popup).add_to(g)

In [ ]:
# Pour sauvegarder la carte
map.save(outfile = 'map.html')

In [55]:
# Pour afficher la carte
# f